**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 10:03:05,631 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 10:03:05.632 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-09 10:03:05.730 | INFO     | __main__:<module>:13 - User Role: reviewer
2025-02-09 10:03:05.733 | INFO     | __main__:<module>:24 - Creating account with name: quizzical_noyce
2025-02-09 10:03:05.735 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:03:05.758 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 10:03:05.760 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:03:05.761 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:03:05.763 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"


2025-02-09 10:03:05,788 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:03:05,790 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:03:06,945 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:03:06,955 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:03:06.960 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 14
2025-02-09 10:03:06.961 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-09 10:03:06.978 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:03:06,981 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:03:06,998 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:03:07,000 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:03:09,986 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:03:09,995 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:03:09,997 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '6957aef5df66fac5ec8f649afd7f7cb4c4174e6f5b3749c451af813cfbbbc7e0', 'timestamp': 1739095389997}
2025-02-09 10:03:09.999 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:03:10,004 - INFO - Appended data under 'quizzical_noyce@test' in logs/account_data.json
2025-02-09 10:03:10.005 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 10

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 10:03:10,056 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 10:03:10.057 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-09 10:03:10.114 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:03:10.135 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 10:03:10.136 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 10:03:10.137 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:03:10.139 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-09 10:03:10,141 - INFO - Processing account: quizzical_noyce@test
2025-02-09 10:03:10,142 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Quizzical Noyce', 'foaf:mbox': 'quizzical_noyce@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Balochistan University of Informati

2025-02-09 10:03:10,175 - INFO - User Account Metadata CID: QmT95GSsoC2BEJQPLGu3SmDJKo8ohsKmfSpUe95RL4ojRx
2025-02-09 10:03:10.176 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-09 10:03:10,191 - INFO - None
2025-02-09 10:03:10,194 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:03:13,020 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:03:13,028 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:03:13.029 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 10:03:13.049 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 10:03:13.050 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                            

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-02-09 10:03:13.070 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:03:13,074 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:03:13,091 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:03:13,093 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:03:16,053 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:03:16,064 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:03:16,066 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '53859623938349461573574a99cd88e47255cfdc80fc8b5dbd5cb736f3ab89ac', 'timestamp': 1739095396066}
2025-02-09 10:03:16.068 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:03:16,071 - INFO - Appended data under 'quizzical_noyce@test' in logs/account_data.json
2025-02-09 10:03:16.072 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 10

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-09 10:03:16.125 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'quizzical_noyce@test', 'schema:publicKey': 'a1b3bfd96e831c024f8bb9bb1572f40b9e39c019db14c87a856865207f4a55e3', 'schema:roleName': 'reviewer'}, 'foaf:mbox': 'quizzical_noyce@email.com', 'foaf:name': 'Quizzical Noyce', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Balochistan University of Information Technology & Management Sciences '}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '3145-3604-8462-X'}}
